In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model 
import pickle 
import numpy as np 
import pandas as pd

In [2]:
## Load the trained model, scaler, pickle, and onehot 

model = load_model('model.h5')

with open('onehot_encoder_geo.pkl', 'rb') as file: 
    label_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file: 
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file: 
    scaler = pickle.load(file)

2025-04-16 09:45:22.266645: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-04-16 09:45:22.266673: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-04-16 09:45:22.266680: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-04-16 09:45:22.266709: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-16 09:45:22.266722: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
# Example input data
input_data = {
    'credit_score': 600,
    'country': 'France',
    'gender': 'Male',
    'age': 40,
    'tenure': 3,
    'balance': 60000,
    'products_number': 2,
    'credit_card': 1,
    'active_member': 1,
    'estimated_salary': 50000
}

In [5]:
# One-hot encode 'Geography'
geo_encoded = label_encoder_geo.transform([[input_data['country']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['country']))
geo_encoded_df

/Users/saivineethgrandhi/Documents/C/GenAI/SimpleANNImplementation/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,country_France,country_Germany,country_Spain
0,1.0,0.0,0.0


In [6]:
input_df = pd.DataFrame([input_data])
input_df

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary
0,600,France,Male,40,3,60000,2,1,1,50000


In [7]:
## Encode the categorical var 

input_df['gender'] = label_encoder_gender.transform(input_df['gender'])
input_df 

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary
0,600,France,1,40,3,60000,2,1,1,50000


In [8]:
## Concatination of OHE data and input df 

input_df = pd.concat([input_df.drop('country', axis= 1), geo_encoded_df], axis= 1)
input_df

,credit_score,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,country_France,country_Germany,country_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [9]:
## Scaling the input data 

input_scaled = scaler.transform(input_df)

In [10]:
## predict 

prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 89ms/step


2025-04-16 09:47:39.917720: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


array([[0.00036313]], dtype=float32)

In [11]:
prediction_prob = prediction[0][0]
prediction_prob

0.0003631318

In [12]:
if prediction_prob >= 0.5: 
    print('Likely to churn')
else: 
    print('Not likely to churn')

Not likely to churn
